In [4]:
from model.UNet import UNet
from model.DeepLab import DeepLabV3Plus
from model.HRNet import HRNetV2

import os

import cv2

import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets, transforms
import torchvision.transforms.functional as TF
from torch.utils.data import Dataset, DataLoader

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

cuda


In [6]:
class SegmentationDataset(Dataset):
  def __init__(self, data_type, mode):    
    self.mode = mode
    self.input, self.label = [], []

    # 작업할 폴더의 경로
    path = './dataset/{0}s'.format(data_type) 

    self.input_path = '{0}/{1}/{2}'.format(path, self.mode, "data")
    self.label_path = '{0}/{1}/{2}'.format(path, self.mode, "label")

    # 폴더 안에 있는 데이터 파일들의 이름을 추출
    data_names = [name.split('.')[0] for name in os.listdir(self.input_path)]

    # 데이터 전처리
    for data_name in data_names:
        self.input.append('{0}/{1}.png'.format(self.input_path, data_name))
        self.label.append('{0}/{1}.png'.format(self.label_path, data_name))

  def __len__(self):
    return len(self.input)

  def __getitem__(self, index):
    input = cv2.cvtColor(cv2.imread(self.input[index]), cv2.COLOR_BGR2RGB)
    label = cv2.imread(self.label[index], cv2.IMREAD_GRAYSCALE)

    input = torch.FloatTensor(input).permute(2, 0, 1)
    label = torch.LongTensor(label)

    i, j, h, w = transforms.RandomCrop.get_params(input, output_size=(512, 512))

    input = TF.crop(input, i, j, h, w)
    label = TF.crop(label, i, j, h, w)
    
    return input, label

In [7]:
train_dataset   = SegmentationDataset('building', 'train')
test_dataset    = SegmentationDataset('building', 'test')

In [8]:
# 저장된 모델 가중치 로드
#model.load_state_dict(torch.load(os.path.join(config["output_dir"], config["model_name"]))

In [9]:
output_dir_path = "./trained"

def train(model, dataset, num_epoch = 10):
    dataloader = DataLoader(dataset, batch_size=8, shuffle=True) 

    loss_func = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters()) 

    for epoch in range(num_epoch):
        model.train()

        costs = []
        
        for batch in dataloader:
            input, label = batch

            input = input.to(device)
            label = label.to(device)

            hypothesis = model(input)
            
            cost = loss_func(hypothesis, label)

            optimizer.zero_grad()
            cost.backward()
            optimizer.step()

            costs.append(cost.item())

        print("{0}/{1} : {2}".format(epoch + 1, num_epoch, np.mean(costs)))
        
        #torch.save(model.state_dict(), os.path.join(output_dir_path, "{0}_epoch_{1:d}.pt".format(type(model).__class__.__name__, epoch + 1)))


In [ ]:
num_epoch = 100

#model1 = UNet().to(device)
model2 = DeepLabV3Plus(num_classes=2).to(device)
#model3 = HRNetV2(num_classes=1).to(device)

#train(model1, train_dataset, num_epoch)
train(model2, train_dataset, num_epoch)
#train(model3, train_dataset, num_epoch)